In [1]:
import pandas as pd 
import numpy as np
import os 
from isswrapper.util.helpers import read_parquet_into_dataframe
from bs4 import BeautifulSoup
import httpx
from isswrapper.loaders.securities import security_description
from urllib.parse import urljoin
from fuzzywuzzy import fuzz
import bs4
import re
from pnd_moex.general.scraper import get_smartlab_forum_urls
import httpx


In [2]:
# loading data
current_path = os.getcwd()
project_path = os.path.dirname(current_path)
datasets_folder_path = os.path.join(project_path, 'datasets')
# news 
pnd_token_date_df = read_parquet_into_dataframe(os.path.join(datasets_folder_path, "pnd_token_date.parquet"))
# all time series
ts_df = read_parquet_into_dataframe(os.path.join(datasets_folder_path, "time_series_securities_pnd.parquet"))
news_df = read_parquet_into_dataframe(os.path.join(datasets_folder_path, "final_df.parquet"))
sn_df = pd.read_parquet(os.path.join(datasets_folder_path, "sec_names.parquet"))

print(ts_df.shape)
ts_df.sample(5)

(135031, 24)


,BOARDID,TRADEDATE,SHORTNAME,SECID,NUMTRADES,VALUE,OPEN,LOW,HIGH,LEGALCLOSEPRICE,...,MARKETPRICE3,ADMITTEDQUOTE,MP2VALTRD,MARKETPRICE3TRADESVALUE,ADMITTEDVALUE,WAVAL,TRADINGSESSION,CURRENCYID,TRENDCLSPR,currencyid
1605,TQBR,2020-10-22,ТНСэнРст,RTSB,15.0,174150.0,0.54,0.52,0.54,0.525,...,0.53,0.525,0.0,501750.0,0.0,0.0,3,SUR,-2.78,RUB
1483,TQBR,2020-04-28,АбрауДюрсо,ABRD,195.0,1589070.0,132.50,131.50,132.50,132.000,...,132.50,132.000,1589070.0,1589070.0,1589070.0,NaN,3,SUR,-0.75,RUB
1202,TQBR,2019-03-18,ВолгЭнСб-п,VGSBP,80.0,228284.0,1.23,1.22,1.38,1.306,...,1.26,1.306,0.0,542196.0,0.0,NaN,3,SUR,6.53,RUB
878,TQBR,2017-12-04,ОКС ао,UCSS,1.0,112000.0,1120.00,1120.00,1120.00,1120.000,...,1120.00,1120.000,1122145.0,898145.0,1122145.0,NaN,3,SUR,0.00,RUB
1840,TQBR,2021-09-24,ИКРУСС-ИНВ,RUSI,7.0,3960.0,56.20,56.20,56.70,56.600,...,56.10,56.600,0.0,503055.0,0.0,0.0,3,SUR,0.53,RUB


### smartlab stuff


In [6]:
base_url = "https://smart-lab.ru/"

In [5]:
client = httpx.Client(base_url=base_url)

In [ ]:
resp = client.get("forum/sectors")
soup = BeautifulSoup(resp.text, "html.parser")
soup

In [7]:
np.random.randint(0, 40, 5)

array([ 0, 31, 13, 12, 18])

In [8]:
tokens = pnd_token_date_df["token"].unique().tolist()
result = get_smartlab_forum_urls([tokens[i] for i in np.random.randint(0, len(tokens), 5)])
result

[('RLMNP', '/forum/RLMN'),
 ('BLNG', '/forum/BLNG'),
 ('PAZA', '/forum/PAZA'),
 ('ABRD', '/forum/ABRD'),
 ('VGSBP', '/forum/VGSB')]

next step is to get last page number and generate url for all pages. Pages on smartlab works as /forum/{security name}/page{numpage}
but first step is to extract number of last page. On default if you go to specified forum page you ll be on the lastmost page. So we need to go through all links and take page numbers, then create for every thread list filled with pages

In [10]:
client = httpx.Client(base_url=base_url)

In [17]:
pre_f_df = []
for token, url in result:
    page_list = generate_forum_thread_page_urls(url)
    url_df = pd.DataFrame({"token":[token]*len(page_list), "url":page_list})
    pre_f_df.append(url_df)
f_df = pd.concat(pre_f_df)
f_df.sample(5)

,token,url
18,ABRD,https://smart-lab.ru/forum/ABRD/page19
0,VGSBP,https://smart-lab.ru/forum/VGSB/page1
0,RLMNP,https://smart-lab.ru/forum/RLMN/page1
24,ABRD,https://smart-lab.ru/forum/ABRD/page25
3,RLMNP,https://smart-lab.ru/forum/RLMN/page4


In [21]:
client = httpx.Client()
resp = client.get("https://smart-lab.ru/forum/ABRD/page19")


In [23]:
str(resp.url)

'https://smart-lab.ru/forum/ABRD/page19'

In [19]:
f_df["url"]

0    https://smart-lab.ru/forum/RLMN/page1
1    https://smart-lab.ru/forum/RLMN/page2
2    https://smart-lab.ru/forum/RLMN/page3
3    https://smart-lab.ru/forum/RLMN/page4
4    https://smart-lab.ru/forum/RLMN/page5
                     ...                  
3    https://smart-lab.ru/forum/VGSB/page4
4    https://smart-lab.ru/forum/VGSB/page5
5    https://smart-lab.ru/forum/VGSB/page6
6    https://smart-lab.ru/forum/VGSB/page7
7    https://smart-lab.ru/forum/VGSB/page8
Name: url, Length: 67, dtype: object

In [ ]:
soup = BeautifulSoup(resp.text,"html.parser")
soup

In [110]:
last_page = int(soup.find("span", class_="page active").text)
pages = [urljoin(base_url, example+f"/page{i}") for i in range(1, last_page+1)]
pages[:5]

['https://smart-lab.ru/forum/BELU/page1',
 'https://smart-lab.ru/forum/BELU/page2',
 'https://smart-lab.ru/forum/BELU/page3',
 'https://smart-lab.ru/forum/BELU/page4',
 'https://smart-lab.ru/forum/BELU/page5']

In [109]:
resp2 = client.get(pages[0])
resp2.status_code

200

In [113]:
result[-5][1]

'/forum/VRSB'

In [116]:
to_parse = generate_forum_thread_page_urls(result[-15][1])

In [128]:
url = to_parse[5]

url

'https://smart-lab.ru/forum/TGKB/page6'

In [122]:
client = httpx.Client()

In [123]:
response=client.get(url)
response.status_code

In [125]:
soup = BeautifulSoup(response.text, "html.parser")

In [126]:
soup

<!DOCTYPE html>

<html lang="ru">
<head>
<title>Форум акции ТГК-2 (TGKB, TGKBP), страница 6</title>
<meta content="text/html; charset=utf-8" http-equiv="content-type"/>
<meta content="width=device-width, initial-scale=1" name="viewport"/>
<meta content="Обсуждение и комментарии инвесторов по акциям ТГК-2. Прогноз курса акций ТГК-2 страница 6" name="DESCRIPTION"> <meta content="ТГК-2, обсуждение компании, информация о компании, новости компании." name="KEYWORDS"/> <meta content="69df339e9279f161" name="yandex-verification"/>
<meta content="17fde70f-5d0a-4de9-809e-a9f5334ce8dd" name="PartnerFinam"/>
<link href="https://mc.yandex.ru" rel="preconnect"/>
<link href="//counter.yadro.ru" rel="preconnect"/>
<link href="https://www.googletagmanager.com" rel="preconnect"/>
<link href="https://yastatic.net" rel="dns-prefetch"/>
<link href="https://content.mql5.com" rel="dns-prefetch"/>
<meta content="no" name="push-subscribes"/>
<meta content="353607944842117" property="fb:app_id"><meta content="

In [131]:
comments = soup.find_all("li", attrs={"data-type":"comment"})

In [172]:
type(comment)

bs4.element.Tag

In [48]:
[np.random.choice(tokens) for i in range(5)]


['LNZLP', 'IDVP', 'BELU', 'TGKN', 'MISB']

In [180]:
comment = comments[0]
comment

<li class="cm_wrap bluid_31628" data-id="10645580" data-type="comment"><a name="comment10645580"></a><a aria-label="Профиль Victor Glukhov" href="/profile/Rbaker/" rel="nofollow"><img alt="Аватар Victor Glukhov" class="avtr_box" loading="lazy" src="/uploads/2021/images/03/16/28/2021/10/11/avatar_100x100.webp?4398"/></a><div class="cmt_body"><div><span><a class="a_name trader_other" href="/profile/Rbaker/" rel="nofollow">Victor Glukhov</a></span><a class="a_time" href="/forum/TGKB/goto_comment_10645580/#comment10645580" rel="nofollow" target="_blank"><time datetime="2020-02-13T11:21:11+03:00">13 февраля 2020, 11:21</time></a></div><div class="text">С июля 2018-ого, до февраля 2020-ого цена почти не менялась.<br/>
Если смотреть на графике (месяц) — всё это выглядит как прямая нитка. Вот и долгожданное сильное движение цены.<br/>
Такая же история была перед ростов в 2017-ом.</div><div class="cm_ftr"><a class="reply" href="/login/" rel="nofollow">Ответить</a><a class="cm_ico th up" href="/

In [138]:
cmt_body = comment.find("div", class_="cmt_body")
cmt_body

<div class="cmt_body"><div><span><a class="a_name trader_other" href="/profile/Malikeldjebena/" rel="nofollow">Malik</a></span><a class="a_time" href="/forum/TGKB/goto_comment_9738142/#comment9738142" rel="nofollow" target="_blank"><time datetime="2019-05-20T16:53:54+03:00">20 мая 2019, 16:53</time></a></div><div class="text"><blockquote class="reply"><blockquote class="reply"><br/>
«ТГК-2» — около 4 млрд рублей.<br/>
</blockquote><br/>
редактор Боб, наверное, все-таки огк-2</blockquote><br/>
bayad, для аналитегов политизированной помойки финанза это одно и то же )</div><div class="cm_ftr"><a class="reply" href="/login/" rel="nofollow">Ответить</a><a class="cm_ico th up" href="/login/" rel="nofollow"></a><a class="cm_ico th dn" href="/login/" rel="nofollow"></a><a class="cm_mrk" href="/login/" rel="nofollow"> 0</a></div></div>

In [140]:
cmt_text = cmt_body.find("div", class_="text")
likes = cmt_body.find("div", class_="cm_ftr")
likes

<div class="cm_ftr"><a class="reply" href="/login/" rel="nofollow">Ответить</a><a class="cm_ico th up" href="/login/" rel="nofollow"></a><a class="cm_ico th dn" href="/login/" rel="nofollow"></a><a class="cm_mrk" href="/login/" rel="nofollow"> 0</a></div>

In [142]:
cmt_text.text

'\r\n«ТГК-2» — около 4 млрд рублей.\n\r\nредактор Боб, наверное, все-таки огк-2\r\nbayad, для аналитегов политизированной помойки финанза это одно и то же )'

In [149]:
int(likes.find("a", class_="cm_mrk").text)

0

In [175]:
cmt_body.find("a", class_="cm_mrk").text

' 0'

In [33]:
import datetime
import arrow
import bs4

In [160]:
cmt_time = arrow.get(cmt_body.find("time").get("datetime")).datetime
cmt_time

datetime.datetime(2019, 5, 20, 16, 53, 54, tzinfo=tzoffset(None, 10800))

In [167]:
user_id = cmt_body.find("a", class_="a_name trader_other").get("href").split("/")[2]
user_id

'Malikeldjebena'

In [171]:
entitiled = not cmt_body.find("a", class_="image_true") is None
entitiled

False

In [41]:
def extract_comment_data(comment: bs4.element.Tag) -> dict:
    """
    Extract valuable data from a comment and package it into a dictionary.

    :param comment: The comment element.
    :type comment: bs4.element.Tag
    :return: Extracted data as a dictionary.
    :rtype: dict
    """
    comment_body = comment.find("div", class_="cmt_body")
    return dict(
        comment_text = comment_body.find("div", class_="text").text,
        comment_score = int(comment_body.find("a", class_="cm_mrk").text.replace(" ", "")),
        comment_datetime = arrow.get(comment_body.find("time").get("datetime")).datetime,
        user_id = comment_body.find("a", class_="a_name trader_other").get("href").split("/")[2],
        badges = not comment_body.find("a", class_="image_true") is None,
        )

In [27]:

resp.text

'<!DOCTYPE html>\n<html lang="ru">\n\t<head>\n\t\t<title>Форум акции Абрау-Дюрсо (ABRD), страница 19</title>\n\t\t<meta http-equiv="content-type" content="text/html; charset=utf-8" />\n\t\t<meta name="viewport" content="width=device-width, initial-scale=1">\n\t\t<meta name="DESCRIPTION" content="Обсуждение и комментарии инвесторов по акциям Абрау-Дюрсо. Прогноз курса акций Абрау-Дюрсо страница 19"/>\t\t<meta name="KEYWORDS" content="Абрау-Дюрсо, обсуждение компании, информация о компании, новости компании."/>\t\t\t\t<meta name=\'yandex-verification\' content=\'69df339e9279f161\' />\n\t\t<meta name="PartnerFinam" content="17fde70f-5d0a-4de9-809e-a9f5334ce8dd">\n\t\t\n\t\t<link rel="preconnect" href="https://mc.yandex.ru">\n\t\t<link rel="preconnect" href="//counter.yadro.ru">\n\t\t<link rel="preconnect" href="https://www.googletagmanager.com">\n\t\t\t\t<link rel="dns-prefetch" href="https://yastatic.net">\n\t\t\t\t\t\t<link rel="dns-prefetch" href="https://content.mql5.com">\n\t\t\n\t\t

In [28]:
f_df = read_parquet_into_dataframe(os.path.join(datasets_folder_path, "test.parquet"))
f_df.head(5)

,token,url,body
0,ABRD,https://smart-lab.ru/forum/ABRD/page1,"<!DOCTYPE html>\n<html lang=""ru"">\n\t<head>\n\..."
1,ABRD,https://smart-lab.ru/forum/ABRD/page2,"<!DOCTYPE html>\n<html lang=""ru"">\n\t<head>\n\..."
2,ABRD,https://smart-lab.ru/forum/ABRD/page3,"<!DOCTYPE html>\n<html lang=""ru"">\n\t<head>\n\..."
3,ABRD,https://smart-lab.ru/forum/ABRD/page4,"<!DOCTYPE html>\n<html lang=""ru"">\n\t<head>\n\..."
4,ABRD,https://smart-lab.ru/forum/ABRD/page5,"<!DOCTYPE html>\n<html lang=""ru"">\n\t<head>\n\..."


In [29]:
bodys = f_df["body"]
len(bodys)

33

In [46]:
f_df["token"].unique().tolist()

['ABRD', 'PAZA']

In [42]:
pre_fin_list = []
for idx, row in f_df.iterrows():
    soup = BeautifulSoup(row["body"])
    comments = soup.find_all("li", attrs={"data-type":"comment"})
    f_list = []
    for comment in comments:
        f_list.append(extract_comment_data(comment))
    n_df = pd.DataFrame(f_list)
    n_df["token"] = row["token"]
    n_df["url"] = row["url"]
    pre_fin_list.append(n_df)
ffff_df = pd.concat(pre_fin_list)
    

,comment_text,comment_score,comment_datetime,user_id,badges,token,url
0,Павловский автобус - Цена выкупа 6339 руб за о...,0,2021-10-28 09:43:03+03:00,Marek,False,PAZA,https://smart-lab.ru/forum/PAZA/page5
15,"Распределить прибыль ПАО «Абрау – Дюрсо», полу...",0,2021-05-24 16:22:53+03:00,Marek,False,ABRD,https://smart-lab.ru/forum/ABRD/page10
3,"Иван Соловьев, есть доказательства, что это ма...",0,2023-07-10 10:32:00+03:00,IvanSolovev,False,PAZA,https://smart-lab.ru/forum/PAZA/page7
23,Этого напитка не хватает в России… Водяра косо...,0,2023-01-17 15:10:09+03:00,Ds10,False,ABRD,https://smart-lab.ru/forum/ABRD/page16
24,Что за чудеса с этой компанией? Новый биткоин?...,1,2019-09-26 17:59:31+03:00,Andy24pro,False,PAZA,https://smart-lab.ru/forum/PAZA/page2
4,Популярность зарубежного вина достигла рекордн...,0,2023-10-03 07:41:38+03:00,Buterbrod2018,False,ABRD,https://smart-lab.ru/forum/ABRD/page25
8,Имущество «ПАЗ»а давно заложено «ГАЗ»ом в обес...,0,2020-06-14 20:25:58+03:00,WBaffet,False,PAZA,https://smart-lab.ru/forum/PAZA/page4
5,"Когда МСФО 2019 год?\r\nkhornickjaadle, удовле...",0,2020-04-08 06:38:42+03:00,khornickjaadle,False,ABRD,https://smart-lab.ru/forum/ABRD/page4
24,Формулировка решения по первому вопросу повест...,0,2023-01-25 22:55:15+03:00,Marek,False,PAZA,https://smart-lab.ru/forum/PAZA/page7
1,Походу аграрное лобби сдувается… \r\nМой прогн...,0,2020-06-15 19:21:25+03:00,FinOrg,False,ABRD,https://smart-lab.ru/forum/ABRD/page4


In [43]:
ffff_df.to_parquet("penis.parquet")

In [39]:
n_df["token"] = 
n_df.head()

,comment_text,comment_score,comment_datetime,user_id,badges
0,"Auximen, а её вообще дают шортить?",0,2017-07-02 13:49:10+03:00,romanranniy,False
1,"Я бы зашортил с текущей цены до 150, но ABRD «...",0,2017-07-02 13:47:41+03:00,Auximen,False
2,"сегодня ожидаем: ABRD: отсечка по д-дам 1,29 р...",0,2017-07-02 00:30:00+03:00,dhc,False
3,"сегодня ожидаем: ABRD: посл. день с дивид. 1,2...",0,2017-06-28 00:30:01+03:00,dhc,False
4,"Ну вот, а сегодня в обратную сторону.",0,2017-06-22 17:54:49+03:00,dimalar,False


In [183]:
decompose_comment(comment)

{'comment_text': 'С июля 2018-ого, до февраля 2020-ого цена почти не менялась.\r\nЕсли смотреть на графике (месяц) — всё это выглядит как прямая нитка. Вот и долгожданное сильное движение цены.\r\nТакая же история была перед ростов в 2017-ом.',
 'comment_score': 0,
 'comment_datetime': datetime.datetime(2020, 2, 13, 11, 21, 11, tzinfo=tzoffset(None, 10800)),
 'user_id': 'Malikeldjebena',
 'badges': False}

### mfd stuff

In [106]:
base_url="https://lite.mfd.ru"

In [ ]:
tokens = pnd_token_date_df["token"]
t_names = [get_sec_names(token) for token in tokens]
t_names
# retry when moex is up

In [8]:
df = pd.DataFrame(t_names)
df.columns = ["shortname", "longname"]
df["token"] = tokens


In [14]:
df.to_parquet(os.path.join(datasets_folder_path, "sec_names.parquet"))

In [108]:
token = np.random.choice(tokens)

In [5]:
def get_mfd_thread_links(tokens):
    # get most visited thread for given token(s)
    result = dict()
    with httpx.Client(base_url="https://lite.mfd.ru") as client:
        for token in tokens:
            response = client.get("/forum/subforum/", params=dict(id=1, q=token))
            soup = BeautifulSoup(response.text, "html.parser")
            search_result_table = soup.find("table", class_="mfd-table mfd-threads")
            wrapped_link = search_result_table.find("td", class_="mfd-item-subject")
            link=None
            if wrapped_link:
                link = wrapped_link.find("a").get("href")
            # wrapped_links = search_result_table.find("td", class_="mfd-item-subject")
            # links = [link.find("a").get("href") for link in wrapped_links]
            result[token]=link
    return result
# shortname and long name usage required
# a lot to go, a lot to do

In [109]:
link_lists = get_mfd_thread_links(tokens)
link_lists

{'KCHE': '/forum/thread/?id=63489&2022',
 'SVET': '/forum/thread/?id=111519&2022',
 'DVEC': '/forum/thread/?id=62197&2022',
 'RTSB': None,
 'RKKE': '/forum/thread/?id=61551&2022',
 'SVAV': '/forum/thread/?id=61394&2022',
 'KCHEP': None}

In [7]:
from pnd_moex.general.scraper import generate_forum_thread_page_urls, extract_comments_from_page

In [ ]:
test_urls = generate_forum_thread_page_urls(
    base_url,
    list(link_lists.values())[0],
    last_page=dict(name="a", class_="mfd-paginator-selected"),
    page_fstr="&page={i}",
    zero_index=True
    )
test_urls

In [132]:
def extract_mfd_comment_data(comment: bs4.element.Tag) -> dict:
    """
    Extract valuable data from a comment and package it into a dictionary.
    :param comment: The comment element.
    :type comment: bs4.element.Tag
    :return: Extracted data as a dictionary.
    :rtype: dict
    """
    comment_misc = comment.find("div", class_="mfd-post-remark")
    user_id = comment.find("a", class_="mfd-poster-link")
    comment_score = comment.find("span", class_="u")
    user_score = comment.find("div", class_="mfd-poster-info-rating mfd-icon-profile-star")
    comment_text = comment.find("div", class_="mfd-post-text")
    return dict(
        comment_text=None if comment_text is None else comment_text.text,
        comment_score=0 if comment_score is None else int(comment_score.text),
        comment_datetime=arrow.get(comment.find("a", class_="mfd-post-link").text, 'DD.MM.YYYY HH:mm').datetime,
        comment_misc = None if comment_misc is None else comment_misc.text,
        user_id = None if user_id is None else user_id.get("href"),
        user_score = None if user_score is None else int(re.search(r'\((\d+)\)',user_score.find("a").get("title")).group(1)),
    )


In [111]:
proba_url = test_urls[0]
proba_url

'https://lite.mfd.ru/forum/thread/?id=63489&2022&page=0'

In [120]:
with httpx.Client() as client:
    resp = client.get(proba_url)

In [133]:
af_df = extract_comments_from_page(resp, dict(name="div", class_="mfd-post"), extract_mfd_comment_data)
af_df.sample(5)

,comment_text,comment_score,comment_datetime,comment_misc,user_id,user_score
0,"Братья и сестры рад приветствовать вас, вот ре...",0,2010-11-26 12:56:00+00:00,Сообщение перенесено администратором МФД 29.11...,None,NaN
12,"Вам на рынке важней брать как вам кажется ""хор...",0,2010-11-28 18:38:00+00:00,None,/forum/poster/?id=59135,2522.0
15,Моя родина - свобода. @ 28.11.2010 18:44Спаси...,0,2010-11-28 18:49:00+00:00,None,/forum/poster/?id=58862,43504.0
14,Спасибо! не пробьет выйдем! Удачи!,0,2010-11-28 18:44:00+00:00,None,/forum/poster/?id=59135,2522.0
2,СПРАВКА: ОАО «Камчатскэнерго» создано в 1964...,0,2010-11-28 17:15:00+00:00,None,/forum/poster/?id=59135,2522.0


In [124]:
soup = BeautifulSoup(resp.text, "html.parser")
tmp = soup.find_all(**dict(name="div", class_="mfd-post"))[4]


In [118]:
extract_mfd_comment_data(tmp)


{'comment_text': 'И последнее добавляю с сайта решение собрания акционеров:  ПРОЕКТЫ РЕШЕНИЙ ВНЕОЧЕРЕДНОГО ОБЩЕГО СОБРАНИЯ АКЦИОНЕРОВ   ОАО «КАМЧАТСКЭНЕРГО»   Вопрос №1: О реорганизации Открытого акционерного общества энергетики и электрификации «Камчатскэнерго» в форме присоединения к нему Открытого акционерного общества «Камчатские ТЭЦ», Открытого акционерного общества «Центральные электрические сети», Открытого акционерного общества «Камчатская тепло-транспортная компания»    Проект решения:  1. Реорганизовать Открытое акционерное общество энергетики и электрификации «Камчатскэнерго» в форме присоединения к нему Открытого акционерного общества «Камчатские ТЭЦ», Открытого акционерного общества «Центральные электрические сети», Открытого акционерного общества «Камчатская тепло-транспортная компания» на условиях, предусмотренных Договором о присоединении Открытого акционерного общества «Камчатские ТЭЦ», Открытого акционерного общества «Центральные электрические сети», Открытого акционе

In [98]:
tmp.find("table")


<table><tr><td><a href="/forum/poster/?id=65375" rel="nofollow">Оптовик</a><td>+</td></td></tr></table>

In [127]:
import re

In [130]:
int(match.group(1))

'2522'

In [86]:
tmp.find("span", class_="u").text

AttributeError: 'NoneType' object has no attribute 'text'

In [79]:
tmp.find("div",class_="mfd-post-top-2").text


'1Оптовик+'

In [70]:
int(tmp.find("div", class_="mfd-post-body-left").text)

ValueError: invalid literal for int() with base 10: ''

In [60]:
tmp.find("div", class_="mfd-post-text").text

'на чем растет???'

In [61]:
tmp.find("div", class_="mfd-post-remark").text

AttributeError: 'NoneType' object has no attribute 'text'

In [ ]:
extract_comments_from_page(resp, comment_dict=dict(name="div", class_="mfd-post"), )

In [1]:
import numpy as np 
import matplotlib.pyplot as plt

In [ ]:
t = np.linspace()

### e-disclosure stuff


In [3]:
tokens = pnd_token_date_df["token"].unique().tolist()

In [54]:
client = httpx.Client(base_url="https://lite.mfd.ru")

In [128]:
res = dict()
for token in tokens:
    response = client.get("/forum/subforum/", params=dict(id=1, q=token))
    soup = BeautifulSoup(response.text, "html.parser")
    tmp = soup.find("td", class_="mfd-item-subject")
    if tmp:
        res[token]=tmp.find("a").text
res

{'ABRD': 'Абрау-Дюрсо (ABRD)',
 'ALBK': 'Бест Эффортс Банк / Best Efforts Bank (ALBK)',
 'ASSB': 'Астраханская энергосбытовая компания (ASSB)',
 'BELU': 'НоваБев Групп, б. Белуга, Синергия (BELU)',
 'BLNG': 'Белон (BLNG)',
 'DASB': 'Дагестанэнергосбыт (DASB)',
 'DVEC': 'Дальневосточная Энергетическая Компания (ДЭК, DVEC)',
 'DZRD': 'Донской завод радиодеталей (DZRD)',
 'ELTZ': 'Электроцинк (ELTZ)',
 'FXRW': 'ETF FXRW',
 'GECO': 'Генетико ЦГРМ (GECO)',
 'GTLC': 'GTL (ГТЛ, GTLC)',
 'GTRK': 'Глобалтрак Менеджмент (ГТМ) (GTRK)',
 'IGST': 'Ижсталь (IGST)',
 'ISKJ': 'Артген биотех (б. ИСКЧ) (ISKJ)',
 'KCHE': 'Камчатскэнерго (KCHE)',
 'KROT': 'Красный октябрь (KROT)',
 'KUBE': 'Россети Кубань, б. Кубаньэнерго (KUBE)',
 'KUZB': 'Банк Кузнецкий (KUZB)',
 'LENT': 'Лента (LENT)',
 'LNZL': 'Лензолото (LNZL)',
 'LVHK': 'Левенгук (LVHK)',
 'MERF': 'Меридиан (MERF)',
 'MGVM': 'Медиа группа «Война и Мир» (MGVM, МедиаВиМ)',
 'MOBB': 'МосОблБАНК (MOBB)',
 'MRKS': 'Россети Сибирь (б. МРСК Сибири) (MRKS)'

a bit of explanation needed. This names are getted from mfd forum. Structure is quite simple, name + alt anme + (token + alt token). Token and alt token are always in brackets, alt names splitted with / sign. so here we have it

In [129]:
res_predf = []
for token, alt_name in res.items():
    tmp_alt_names = []
    tmp_list = alt_name.split(" (")
    tmp_alt_names.extend(tmp_list[0].split(" / "))
    for i in range(1, len(tmp_list)):
        tmp_alt_names.extend(tmp_list[i][:-1].split(", "))
    res_predf.append((token, tmp_alt_names))
res_predf

[('ABRD', ['Абрау-Дюрсо', 'ABRD']),
 ('ALBK', ['Бест Эффортс Банк', 'Best Efforts Bank', 'ALBK']),
 ('ASSB', ['Астраханская энергосбытовая компания', 'ASSB']),
 ('BELU', ['НоваБев Групп, б. Белуга, Синергия', 'BELU']),
 ('BLNG', ['Белон', 'BLNG']),
 ('DASB', ['Дагестанэнергосбыт', 'DASB']),
 ('DVEC', ['Дальневосточная Энергетическая Компания', 'ДЭК', 'DVEC']),
 ('DZRD', ['Донской завод радиодеталей', 'DZRD']),
 ('ELTZ', ['Электроцинк', 'ELTZ']),
 ('FXRW', ['ETF FXRW']),
 ('GECO', ['Генетико ЦГРМ', 'GECO']),
 ('GTLC', ['GTL', 'ГТЛ', 'GTLC']),
 ('GTRK', ['Глобалтрак Менеджмент', 'ГТМ', 'GTRK']),
 ('IGST', ['Ижсталь', 'IGST']),
 ('ISKJ', ['Артген биотех', 'б. ИСКЧ', 'ISKJ']),
 ('KCHE', ['Камчатскэнерго', 'KCHE']),
 ('KROT', ['Красный октябрь', 'KROT']),
 ('KUBE', ['Россети Кубань, б. Кубаньэнерго', 'KUBE']),
 ('KUZB', ['Банк Кузнецкий', 'KUZB']),
 ('LENT', ['Лента', 'LENT']),
 ('LNZL', ['Лензолото', 'LNZL']),
 ('LVHK', ['Левенгук', 'LVHK']),
 ('MERF', ['Меридиан', 'MERF']),
 ('MGVM', ['Ме

In [130]:
new_sn_df = pd.merge(sn_df, pd.DataFrame(res_predf, columns=["token", "alt_names"]), on="token",how="left")
new_sn_df.sample(5)

,token,shortname,longname,alt_name_1,alt_names
21,KCHEP,КамчатЭ ап,Камчатскэнерго ПАО ап,Камчатскэнерго,NaN
33,LVHK,Левенгук,Левенгук ПАО ао,левенгук,"[Левенгук, LVHK]"
34,MERF,МЕРИДИАН,МЕРИДИАН ПАО ао,МЕРИДИАН,"[Меридиан, MERF]"
39,MRKS,РсетСиб ао,Россети Сибирь ао,Россети Сибирь,"[Россети Сибирь, б. МРСК Сибири, MRKS]"
41,MSTT,Мостотрест,"ПАО ""МОСТОТРЕСТ"" ао",Мостотрест,"[Мостотрест, MSTT]"


In [131]:
new_sn_df.head(5)

,token,shortname,longname,alt_name_1,alt_names
0,ABRD,АбрауДюрсо,Абрау-Дюрсо ПАО ао,Абрау-Дюрсо,"[Абрау-Дюрсо, ABRD]"
1,ALBK,БестЭфБ ао,Бест Эффортс Банк ПАО ао,Бест Эффортс Банк,"[Бест Эффортс Банк, Best Efforts Bank, ALBK]"
2,ASSB,АстрЭнСб,"""Астраханская ЭСК"" ПАО",Астраханэнергосбыт,"[Астраханская энергосбытовая компания, ASSB]"
3,BELU,НоваБев ао,НоваБев Групп ПАО ао,НоваБев Групп (Белуга Групп),"[НоваБев Групп, б. Белуга, Синергия, BELU]"
4,BLNG,Белон ао,Белон ОАО ао,Белон,"[Белон, BLNG]"


In [133]:
res = []
for alt_name, alt_name_list in zip(new_sn_df["alt_name_1"].tolist(), new_sn_df["alt_names"].tolist()):
    if alt_name is None:
        res.append(None if type(alt_name_list) != list else alt_name_list)
        continue
    if type(alt_name_list) != list:
        res.append([alt_name])
        continue
    alt_name_list.append(alt_name)
    res.append(alt_name_list)
res

[['Абрау-Дюрсо', 'ABRD', 'Абрау-Дюрсо'],
 ['Бест Эффортс Банк', 'Best Efforts Bank', 'ALBK', 'Бест Эффортс Банк'],
 ['Астраханская энергосбытовая компания', 'ASSB', 'Астраханэнергосбыт'],
 ['НоваБев Групп, б. Белуга, Синергия',
  'BELU',
  'НоваБев Групп (Белуга Групп)'],
 ['Белон', 'BLNG', 'Белон'],
 ['Банк Санкт-Петербург'],
 ['ЧКПЗ'],
 ['Дагестанэнергосбыт', 'DASB', 'ДЭСК'],
 ['Дальневосточная Энергетическая Компания', 'ДЭК', 'DVEC', 'ДЭК'],
 ['Донской завод радиодеталей', 'DZRD', 'ДЗРД - Донской завод радиодеталей'],
 ['Электроцинк', 'ELTZ', 'Электроцинк'],
 ['ETF FXRW', 'FinEx RUB GLOBAL EQUITY UC ETF'],
 ['FinEx USD GLOBAL EQUITY UC ETF'],
 ['Генетико ЦГРМ', 'GECO', 'Генетико'],
 ['GTL', 'ГТЛ', 'GTLC', 'GTL'],
 ['Глобалтрак Менеджмент', 'ГТМ', 'GTRK', 'Globaltruck (ГТМ)'],
 ['Инвест-Девелопмент'],
 ['Ижсталь', 'IGST', 'Ижсталь'],
 ['Ижсталь'],
 ['Артген биотех', 'б. ИСКЧ', 'ISKJ', 'Артген биотех (ИСКЧ)'],
 ['Камчатскэнерго', 'KCHE', 'Камчатскэнерго'],
 ['Камчатскэнерго'],
 ['Крас

In [134]:
new_sn_df["alt_names"] = res
new_sn_df.drop(columns=["alt_name_1"], inplace=True)
new_sn_df

,token,shortname,longname,alt_names
0,ABRD,АбрауДюрсо,Абрау-Дюрсо ПАО ао,"[Абрау-Дюрсо, ABRD, Абрау-Дюрсо]"
1,ALBK,БестЭфБ ао,Бест Эффортс Банк ПАО ао,"[Бест Эффортс Банк, Best Efforts Bank, ALBK, Б..."
2,ASSB,АстрЭнСб,"""Астраханская ЭСК"" ПАО","[Астраханская энергосбытовая компания, ASSB, А..."
3,BELU,НоваБев ао,НоваБев Групп ПАО ао,"[НоваБев Групп, б. Белуга, Синергия, BELU, Нов..."
4,BLNG,Белон ао,Белон ОАО ао,"[Белон, BLNG, Белон]"
...,...,...,...,...
67,VRSB,ТНСэнВорон,ТНС энерго Воронеж ПАО ао,"[ТНС энерго Воронеж, VRSB, Воронежская энергос..."
68,VRSBP,ТНСэнВор-п,ТНС энерго Воронеж ПАО ап,[ТНС энерго Воронеж]
69,VSYD,ВыбСудЗ ао,Выборгский Суд.Завод ПАО ао,"[Выборгский судостроительный завод, ВСЗ, VSYD,..."
70,VSYDP,ВыбСудЗ ап,Выборгский Суд.Завод ПАО ап,[Выборгский судостроительный завод (ВСЗ)]


In [135]:
new_sn_df.to_parquet(os.path.join(datasets_folder_path, "sec_names.parquet"))

In [95]:

url = 'https://www.e-disclosure.ru/api/search/companies'
q_base = {
    'radReg': 'FederalDistricts',
    'districtsCheckboxGroup': '-1',
    'regionsCheckboxGroup': '-1',
    'branchesCheckboxGroup': '-1',
    'lastPageSize': '10',
    'lastPageNumber': '1',
}

In [96]:
client = httpx.Client()


In [ ]:

finale = dict()
# Выполните POST-запрос
for name_list in sn_df.values.tolist():
    finale[name_list[0]]=[]
    name_list_flatten = pd.Series(name_list).explode().dropna().drop_duplicates().tolist()
    name_list_flatten.sort(key=lambda x: len(x) , reverse=True)
    for name in name_list_flatten:
        q_base["textfield"] = name
        q_base["query"] = name
        response = client.post(url, data=q_base, follow_redirects=True)
        if response.status_code==200 and response.json()["allFoundCompanies"]!=0:
            finale[name_list[0]].extend(response.json()["foundCompaniesList"])

list(finale.values())[:5]


In [4]:
from collections import Counter

In [108]:
processed = dict()
for key, value in finale.items():
    tmp_company_list = Counter()
    for s_pos in value:
        if s_pos["docCount"]!=0:
            tmp_company_list[s_pos["id"]]+=1
    processed[key] = list(tmp_company_list.keys())
processed

{'ABRD': [26517],
 'ALBK': [],
 'ASSB': [5966],
 'BELU': [7380],
 'BLNG': [332],
 'BSPBP': [3935],
 'CHKZ': [3306],
 'DASB': [],
 'DVEC': [6658, 11385, 31124, 19810, 38284],
 'DZRD': [5496],
 'ELTZ': [3521],
 'FXRW': [],
 'FXWO': [],
 'GECO': [38201],
 'GTLC': [31201, 31532],
 'GTRK': [37114],
 'IDVP': [34421],
 'IGST': [4089],
 'IGSTP': [4089],
 'ISKJ': [7814],
 'KCHE': [9004, 9003, 1409, 29921],
 'KCHEP': [9004, 9003, 1409],
 'KRKOP': [222],
 'KROT': [595, 33622, 30191, 22858],
 'KROTP': [],
 'KTSB': [5963, 8349, 1922],
 'KTSBP': [5963],
 'KUBE': [2827],
 'KUZB': [30898],
 'LENT': [38380, 3982, 4240, 19677, 34238],
 'LNZL': [1991],
 'LNZLP': [1991],
 'LPSB': [5082],
 'LVHK': [31319],
 'MERF': [34370, 11061, 9648, 19117, 62, 13756],
 'MGVM': [29626],
 'MISB': [6070],
 'MISBP': [6070],
 'MOBB': [26831],
 'MRKS': [12072],
 'MSST': [31563],
 'MSTT': [29608, 1220],
 'NKHP': [10198, 13386],
 'NNSBP': [7022],
 'ORUP': [31548, 28359, 4085, 21363, 8658, 33614, 37412],
 'PAZA': [3638],
 'RDRB'

In [105]:
c_test = Counter([12233, 332211, 112233, 33321, 12233, 332211, 111333, 12233])
c_test.keys()

dict_keys([12233, 332211, 112233, 33321, 111333])

In [107]:
list(c_test.keys())

[12233, 332211, 112233, 33321, 111333]

In [163]:
processed = dict()
for key, value in finale.items():
    tmp_company_list = set()
    for s_pos in value:
        if s_pos["docCount"]!=0:
            tmp_company_list.add(s_pos["id"])
    processed[key] = list(tmp_company_list)
processed

{'ABRD': [26517],
 'ALBK': [],
 'ASSB': [],
 'BELU': [7380],
 'BLNG': [332],
 'BSPBP': [3935],
 'CHKZ': [3306],
 'DASB': [],
 'DVEC': [38284, 19810, 31124, 11385],
 'DZRD': [5496],
 'ELTZ': [3521],
 'FXRW': [],
 'FXWO': [],
 'GECO': [38201],
 'GTLC': [31201],
 'GTRK': [37114],
 'IDVP': [34421],
 'IGST': [4089],
 'IGSTP': [4089],
 'ISKJ': [7814],
 'KCHE': [1409, 9003, 9004, 29921],
 'KCHEP': [1409, 9003, 9004],
 'KRKOP': [222],
 'KROT': [],
 'KROTP': [],
 'KTSB': [1922, 5963, 8349],
 'KTSBP': [5963],
 'KUBE': [2827],
 'KUZB': [30898],
 'LENT': [38380, 3982, 4240, 19677, 34238],
 'LNZL': [1991],
 'LNZLP': [1991],
 'LPSB': [5082],
 'LVHK': [31319],
 'MERF': [34370, 19117, 9648, 11061, 13756, 62],
 'MGVM': [],
 'MISB': [6070],
 'MISBP': [6070],
 'MOBB': [26831],
 'MRKS': [12072],
 'MSST': [31563],
 'MSTT': [29608, 1220],
 'NKHP': [13386, 10198],
 'NNSBP': [7022],
 'ORUP': [28359, 8658, 21363, 4085, 31548],
 'PAZA': [3638],
 'RDRB': [1398],
 'RKKE': [14659, 1615],
 'RLMNP': [29610],
 'ROSB'

In [116]:
np.linspace(0,1, 1)

array([0.])

In [119]:

def weight_gen(n):
    if n<1:
        raise ValueError("Can't generate negative or zero wheights")
    points = np.ones(n) - np.linspace(0, 1, n)*0.1
    return points
weight_gen(1)

array([1.])

In [ ]:
c= Counter()
c

Counter()

In [120]:
pd.DataFrame(processed.items(), columns=["token", "ids"]).to_parquet(os.path.join(datasets_folder_path, "temporal_data.parquet"))

In [121]:
todo_df = pd.read_parquet(os.path.join(datasets_folder_path, "temporal_data.parquet"))
todo_df.sample(10)

,token,ids
46,RDRB,[1398]
11,FXRW,[]
13,GECO,[38201]
45,PAZA,[3638]
32,LPSB,[5082]
3,BELU,[7380]
27,KUBE,[2827]
1,ALBK,[]
58,TGKN,[9396]
56,SVET,[37412]


In [141]:
token = np.random.choice(tokens)
ptids = todo_df.set_index("token").loc[token,"ids"]
ptids

array([35102,  2658,   533,  8924, 22825, 22814,  8550], dtype=int64)

In [125]:
base_url = "https://www.e-disclosure.ru"
client = httpx.Client(base_url=base_url)

In [14]:
from rapidfuzz import fuzz

In [7]:
import jaro

In [5]:
def company_similarity_check(token, id_list):
    if len(id_list)<1:
        return []
    fuzz_ratio_list = []
    jaro_ratio_list = []
    base_url = "https://www.e-disclosure.ru"
    with httpx.Client(base_url=base_url) as b_client:
        for id_tc, weight in zip(id_list, weight_gen(len(id_list))):
            # acquire lname and s name from e-disclosure
            resp = b_client.get("/portal/company.aspx", params=dict(id=id_tc),follow_redirects=True)
            soup = BeautifulSoup(resp.text, "html.parser")
            t_df = pd.read_html(str(soup.find("table", class_="zebra company-table")))[0]
            names = pd.Series(sn_df.set_index("token").loc[token]).explode().dropna().drop_duplicates().tolist()
            lo_name = t_df.loc[0, 1]
            so_name = t_df.loc[1, 1]
            # similarity ratio via fuzz lib
            similarity_ratio_fuzz = sum(
            [
                max([fuzz.partial_ratio(name, lo_name) for name in names]),
                max([fuzz.ratio(lo_name, name) for name in names]),
                max([fuzz.partial_ratio(name, so_name) for name in names]),
                max([fuzz.ratio(so_name, name) for name in names]),
                ]
            )
            fuzz_ratio_list.append(similarity_ratio_fuzz/400*weight)
            # distnace via jaro lib
            similarity_ratio_jaro = sum(
                [
                    max([jaro.jaro_winkler_metric(name, lo_name) for name in names]),
                    max([jaro.jaro_metric(name, lo_name) for name in names]),
                    max([jaro.jaro_winkler_metric(name, so_name) for name in names]),
                    max([jaro.jaro_metric(name, so_name) for name in names]),
                ]
            )
            jaro_ratio_list.append(similarity_ratio_jaro/4*weight)
    return list(zip(id_list, fuzz_ratio_list, jaro_ratio_list))


In [216]:
todo_df["scores"] = todo_df.apply(lambda row: company_similarity_check(row["token"], row["ids"]), axis=1)


In [217]:
def get_best(scoring:list):
    if len(scoring) == 0:
        return None, None 
    pre_res = [(pos[0], pos[1]+pos[2])for pos in scoring]
    pre_res.sort(key=lambda x: x[1], reverse=True)
    return pre_res[0][0],pre_res[0][1]
def get_best_score(scoring:list, k=1):
    if len(scoring) == 0:
        return None, None 
    scoring.sort(key=lambda x: x[k], reverse=True)
    return scoring[0][0],scoring[0][k] 
todo_df["best_id_overall"] = todo_df["scores"].apply(lambda x: get_best(x)[0])
todo_df["best_score_overall"] = todo_df["scores"].apply(lambda x: get_best(x)[1])
todo_df["best_id_fuzz"] = todo_df["scores"].apply(lambda x: get_best_score(x)[0])
todo_df["best_score_fuzz"] = todo_df["scores"].apply(lambda x: get_best_score(x)[1])
todo_df["best_id_jaro"] = todo_df["scores"].apply(lambda x: get_best_score(x,2)[0])
todo_df["best_score_jaro"] = todo_df["scores"].apply(lambda x: get_best_score(x,2)[1])


In [219]:
todo_df.sample(3)

,token,ids,scores,best_match_id,best_score,best_id_fuzz,best_score_fuzz,best_id_jaro,best_score_jaro,best_id_overall,best_score_overall
65,VJGZ,"[3166, 758]","[(3166, 0.8154761904761905, 0.6756598398107833...",3166.0,1.050504,3166.0,0.815476,3166.0,0.675660,3166.0,1.491136
31,LNZLP,[1991],"[(1991, 0.7580882352941177, 0.7353510226644555)]",1991.0,0.925191,1991.0,0.758088,1991.0,0.735351,1991.0,1.493439
1,ALBK,[],[],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [221]:
todo_df.drop(columns=["best_match_id","best_score"], inplace=True)
todo_df.sample(3)

,token,ids,scores,best_id_fuzz,best_score_fuzz,best_id_jaro,best_score_jaro,best_id_overall,best_score_overall
21,KCHEP,"[9004, 9003, 1409]","[(1409, 0.7060806345009915, 0.6321428571428571...",1409.0,0.706081,1409.0,0.632143,1409.0,1.338223
36,MISB,[6070],"[(6070, 0.8497119078104994, 0.634055330634278)]",6070.0,0.849712,6070.0,0.634055,6070.0,1.483767
2,ASSB,[5966],"[(5966, 0.9021978021978022, 0.7575914423740511)]",5966.0,0.902198,5966.0,0.757591,5966.0,1.659789


In [181]:
todo_df.to_parquet(os.path.join(datasets_folder_path, "temporal_data.parquet"))

In [6]:
todo_df = pd.read_parquet(os.path.join(datasets_folder_path, "temporal_data.parquet"))
todo_df.sample(3)

,token,ids,scores,best_id_fuzz,best_score_fuzz,best_id_jaro,best_score_jaro,best_id_overall,best_score_overall
23,KROT,"[595, 33622, 30191, 22858]","[[595.0, 0.7878787878787878, 0.680739259496258...",595.0,0.787879,595.0,0.680739,595.0,1.468618
25,KTSB,"[5963, 8349, 1922]","[[5963.0, 0.8526739926739926, 0.68060391847087...",5963.0,0.852674,5963.0,0.680604,5963.0,1.533278
71,YAKG,[4994],"[[4994.0, 0.792615709112134, 0.5419434632200589]]",4994.0,0.792616,4994.0,0.541943,4994.0,1.334559


In [7]:
todo_df.sample(5)

,token,ids,scores,best_id_fuzz,best_score_fuzz,best_id_jaro,best_score_jaro,best_id_overall,best_score_overall
59,TNSE,"[33277, 11313, 4717, 10624, 6070, 7022, 6930, ...","[[33277.0, 0.7937368262571515, 0.6888092567062...",33277.0,0.793737,33277.0,0.688809,33277.0,1.482546
18,IGSTP,[4089],"[[4089.0, 0.7494680851063831, 0.65839438339438...",4089.0,0.749468,4089.0,0.658394,4089.0,1.407862
50,ROST,[9038],"[[9038.0, 0.7245638945233266, 0.60920274170274...",9038.0,0.724564,9038.0,0.609203,9038.0,1.333767
41,MSTT,"[29608, 1220]","[[1220.0, 0.7151396889400922, 0.66579865361077...",1220.0,0.715140,1220.0,0.665799,1220.0,1.380938
2,ASSB,[5966],"[[5966.0, 0.9021978021978022, 0.75759144237405...",5966.0,0.902198,5966.0,0.757591,5966.0,1.659789


In [8]:
names = pd.Series(sn_df.set_index("token").loc["UCSS"]).explode().dropna().drop_duplicates().tolist()
names

['ОКС ао', 'ПАО ОКС ао', 'Объединенные Кредитные Системы, ОКС', 'UCSS', 'ОКС']

In [12]:
all([True, True, False])

False

In [ ]:
todo_df["stable"] = (todo_df['col1'].notna()) & (todo_df['col2'].notna()) & (todo_df['col3'].notna()) & (todo_df['col1'] == todo_df['col2']) & (todo_df['col2'] == todo_df['col3'])


In [9]:
todo_df.head(50)

,token,ids,scores,best_id_fuzz,best_score_fuzz,best_id_jaro,best_score_jaro,best_id_overall,best_score_overall
0,ABRD,[26517],"[[26517.0, 0.6253358710255261, 0.6589753368929...",26517.0,0.625336,26517.0,0.658975,26517.0,1.284311
1,ALBK,[],[],NaN,NaN,NaN,NaN,NaN,NaN
2,ASSB,[5966],"[[5966.0, 0.9021978021978022, 0.75759144237405...",5966.0,0.902198,5966.0,0.757591,5966.0,1.659789
3,BELU,[7380],"[[7380.0, 0.6682765151515152, 0.62744239826619...",7380.0,0.668277,7380.0,0.627442,7380.0,1.295719
4,BLNG,[332],"[[332.0, 0.7174744897959183, 0.5498347373347373]]",332.0,0.717474,332.0,0.549835,332.0,1.267309
5,BSPBP,[3935],"[[3935.0, 0.5673830745639257, 0.61358024691358...",3935.0,0.567383,3935.0,0.613580,3935.0,1.180963
6,CHKZ,[3306],"[[3306.0, 0.5357142857142858, 0.58334881081359...",3306.0,0.535714,3306.0,0.583349,3306.0,1.119063
7,DASB,[],[],NaN,NaN,NaN,NaN,NaN,NaN
8,DVEC,"[6658, 11385, 31124, 19810, 38284]","[[6658.0, 0.6573065577490356, 0.59820974820974...",11385.0,0.759375,6658.0,0.598210,11385.0,1.355720
9,DZRD,[5496],"[[5496.0, 0.6358543417366946, 0.56473105438622...",5496.0,0.635854,5496.0,0.564731,5496.0,1.200585


In [151]:
scoring_res = company_similarity_check(ptids)
scoring_res.sort(key=lambda x: x[1], reverse=True)
print("Best score fuzz {:.2f}, id {}".format(scoring_res[0][1], scoring_res[0][0]))
scoring_res.sort(key=lambda x: x[-1], reverse=True)
print("Best score jaro {:.2f}, id {}".format(scoring_res[0][2], scoring_res[0][0]))

Best score fuzz 0.80, id 35102
Best score jaro 0.72, id 35102
